##### CCD : Cvd to close delta

L'objectif de cette indicateur et de faire un oscilateur qui permet de détecter les périodes d'un écart anormale entre le CVD et le prix qui pourrait tout simplement détecter les périodes d'absorption 




##### Methode 1

Z-score
-> Distance euclidienne
-> Corélation
-> Ratio ou différentiel relatif


##### Methode 2

Min-Max Scaling
-> Distance euclidienne
-> Corélation
-> Ratio ou différentiel relatif


#### La suite
Derrière rajouter pourquoi pas un filtre de volatilité avec la vol réalisé long termet et court terme avec un frein appliquer lorsque la volat long terme surpasse celle court
terme 

In [1]:
import numpy as np 
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio  
import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
sys.path.append(function_essential_path)
import utils as us

In [2]:
# Importer la data et la préparé a
data = pd.read_csv(r'H:\Desktop\Data\ES_100T.csv')
print(data)

            open     high      low    close  volume  delta     CVD  \
0        4493.25  4493.50  4491.25  4492.50     354    -58     -58   
1        4492.50  4493.25  4492.25  4492.75     293    -77    -135   
2        4492.75  4494.25  4492.25  4492.50     264    -36    -171   
3        4492.50  4493.50  4492.50  4492.75     146      6    -165   
4        4492.75  4494.00  4492.50  4493.00     185    -11    -176   
...          ...      ...      ...      ...     ...    ...     ...   
1527496  5772.50  5773.50  5771.75  5772.25     254    -58 -151198   
1527497  5772.00  5772.00  5770.75  5771.00     220    -84 -151282   
1527498  5771.00  5771.00  5769.50  5770.75     213    -87 -151369   
1527499  5770.50  5771.00  5769.25  5769.75     227     -5 -151374   
1527500  5770.00  5770.50  5769.75  5770.00     167     -1 -151375   

               Date      Time             Datetime  
0        2022-02-01  00:00:00  2022-02-01 00:00:00  
1        2022-02-01  00:00:11  2022-02-01 00:00:11  


In [22]:
#Préparé la data -> Z-score

mean_cvd = data["CVD"].rolling(window=10).mean()
std_cvd = data["CVD"].rolling(window=10).std()
Z_SCORE_CVD = (data['CVD'] - mean_cvd) / std_cvd

Z_SCORE_CVD = Z_SCORE_CVD.rolling(window=10).mean()


In [23]:
mean_close = data["close"].rolling(window=10).mean()
std_close = data["close"].rolling(window=10).std()
Z_SCORE_close = (data['close'] - mean_close) / std_close

Z_SCORE_close = Z_SCORE_close.rolling(window=10).mean()

In [32]:
window_z_score = Z_SCORE_CVD.iloc[2000:2300]
window_z_score_close = Z_SCORE_close.iloc[2000:2300]
index_num = np.arange(1, len(window_z_score) + 1)


fig = go.Figure(data=[go.Scatter(
    x=index_num, 
    y=window_z_score,
    mode='lines',
    name='Z-score CVD',
    line=dict(color='#00ff88', width=1)
)])

fig.add_trace(go.Scatter(
    x=index_num,
    y=window_z_score_close,
    mode='lines',
    name='Z-score close',
    line=dict(color='#00fff8', width=1)
))
fig.show()

In [33]:
div_ratio = Z_SCORE_close - Z_SCORE_CVD

In [34]:
windows_div_ratio = div_ratio.iloc[2000:2300]
index_num = np.arange(1, len(windows_div_ratio) + 1)
fig = go.Figure(data=[go.Scatter(
    x=index_num, 
    y=windows_div_ratio,
    mode='lines',
    name='div_ratio',
    line=dict(color='#00ff88', width=1)
)])
fig.show()

In [6]:

# Distance euclidienne 


distance_euclidienne = np.sqrt((Z_SCORE_CVD-Z_SCORE_close)**2)
distance_euclidienne = distance_euclidienne.rolling(window=10).mean()

In [16]:

windows_distance_euclidienne = distance_euclidienne.iloc[1700:2000]
index_num = np.arange(1, len(windows_distance_euclidienne) + 1)
fig = go.Figure(data=[go.Scatter(
    x=index_num, 
    y=windows_distance_euclidienne,
    mode='lines',
    name='distance_euclidienne',
    line=dict(color='#00ff88', width=1)
)])
fig.show()

In [8]:
#Ratio / différentiel 

normalized_difference = (Z_SCORE_CVD - Z_SCORE_close) / np.maximum(np.abs(Z_SCORE_CVD), np.abs(Z_SCORE_close))
normalized_difference = normalized_difference.rolling(window=10).mean()
print(normalized_difference)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
1527496   -0.215538
1527497   -0.230397
1527498   -0.279539
1527499   -0.198768
1527500   -0.169513
Length: 1527501, dtype: float64


In [21]:

windows_normalized_difference = normalized_difference.iloc[1700:1800]
index_num = np.arange(1, len(windows_normalized_difference) + 1)
fig = go.Figure(data=[go.Scatter(
    x=index_num, 
    y=windows_normalized_difference,
    mode='lines',
    name='normalised difference',
    line=dict(color='#00ff88', width=1)
)])
fig.show()

In [17]:
#Corélation
window_size = 10  # Définit la taille de la fenêtre glissante

correlation = pd.Series(Z_SCORE_CVD).rolling(window=window_size).corr(pd.Series(Z_SCORE_close))



In [20]:
windows_correlation = correlation.iloc[1700:1800]
index_num = np.arange(1, len(windows_correlation) + 1)
fig = go.Figure(data=[go.Scatter(
    x=index_num, 
    y=windows_correlation,
    mode='lines',
    name='corélation',
    line=dict(color='#00ff88', width=1)
)])
fig.show()